In [1]:
import duckdb
import pandas as pd
import joblib
import json
import plotly.express as px


In [2]:
model = joblib.load("../models/model_v1.pkl")

with open("../models/metrics_v1.json") as f:
    metrics = json.load(f)

con = duckdb.connect("../aquarela.db")
df = con.execute("SELECT * FROM consumo_features").fetchdf()
con.close()

df = df.dropna()
df["date"] = pd.to_datetime(df["date"])

features = ["lag_1", "rolling_mean_7", "temperature", "humidity"]
df["prediction"] = model.predict(df[features])


In [3]:
fig = px.line(
    df,
    x="date",
    y=["consumption_kwh", "prediction"],
    title="Consumo Real vs Previsão - Geral",
)

fig.update_layout(template="plotly_white")
fig.show()


### O modelo consegue capturar bem a tendência geral do consumo, apresentando aderência ao comportamento médio da série. Observa-se uma suavização natural das previsões, típica de modelos lineares, e alguma dificuldade na captura de picos extremos, o que indica oportunidade de melhoria com modelos mais complexos.

# Filtro Interativo por Cliente

In [8]:
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=sorted(df["client_id"].unique()),
    description='Cliente:',
)

def plot_cliente(cliente):
    df_cliente = df[df["client_id"] == cliente]

    fig = px.line(
        df_cliente,
        x="date",
        y=["consumption_kwh", "prediction"],
        title=f"Cliente {cliente} - Real vs Previsão"
    )
    fig.update_layout(template="plotly_white")
    fig.show()

def update_plot(change):
    plot_cliente(change["new"])

dropdown.observe(update_plot, names="value")

display(dropdown)

# Mostrar primeiro automaticamente
plot_cliente(dropdown.value)



Dropdown(description='Cliente:', options=('C0000', 'C0001', 'C0002', 'C0003', 'C0004', 'C0005', 'C0006', 'C000…

### Ao analisar um cliente específico, observamos que o modelo captura bem o nível médio e a tendência temporal do consumo. Entretanto, há dificuldade em reproduzir variações abruptas, o que sugere espaço para modelos mais complexos ou features adicionais para capturar eventos extremos.